In [18]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, row_number
spark = SparkSession.builder.master('local[*]').appName('Analytical_ETL').getOrCreate()

In [19]:
def corrected_parquet_path(date,type):
    basepath='/home/roger/SB/Guided_Capstone'
    filepath='/output/corrected_data/{}/partition={}/'.format(date,type)
    df=spark.read.parquet(basepath+filepath)
    return df

In [20]:
#read parquet files from EOD Corrected data
curr_quote=corrected_parquet_path('2020-08-06','Q')
curr_trade=corrected_parquet_path('2020-08-06','T')
prev_trade=corrected_parquet_path('2020-08-05','T')

In [21]:
#creating temp view
curr_quote.createOrReplaceTempView('tmp_curr_quote')
curr_trade.createOrReplaceTempView('tmp_curr_trade')
prev_trade.createOrReplaceTempView('tmp_prev_trade')

In [22]:
# skip  
# populating the prev_quote data so we can apply analytical functions to current days data

# prev_quote=corrected_parquet_path('2020-08-05','Q')
# prev_quote.createOrReplaceTempView('tmp_prev_quote')

# pprev_ma=spark.sql('SELECT symbol,exchange,event_tm,event_seq_nb,trade_pr, AVG(trade_pr) OVER (PARTITION BY exchange,symbol ORDER BY event_tm RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as mov_avg_pr FROM tmp_prev_trade')
# pprev_ma.createOrReplaceTempView('pprev_ma')
# prev_quote_ma = spark.sql('SELECT q.symbol, q.exchange, q.event_tm, q.event_seq_nb, q.bid_pr, q.bid_size, q.ask_pr, q.ask_size, ma.trade_pr,ma.mov_avg_pr, ROW_NUMBER() OVER (PARTITION BY q.symbol, q.exchange, q.event_tm ORDER BY ma.event_tm DESC) as row_num FROM tmp_prev_quote as q LEFT JOIN pprev_ma as ma ON q.symbol = ma.symbol AND q.exchange = ma.exchange AND ma.event_tm < q.event_tm')

# prev_quote_ma.createOrReplaceTempView('prev_quote_ma')
# final_prev_quote_ma=spark.sql('SELECT symbol, exchange, event_tm, event_seq_nb, bid_pr, bid_size, ask_pr, ask_size, trade_pr as last_trade_pr, mov_avg_pr as last_mov_avg_pr FROM prev_quote_ma where row_num=1')
# final_prev_quote_ma.createOrReplaceTempView('final_prev_ma')

### Calculating Current Day information
#### 30 minute moving average

In [23]:
# 30MA for current days trade information
# uses tmp_curr_trade table
curr_ma_df=spark.sql('SELECT symbol,exchange,event_tm,event_seq_nb, trade_pr,AVG(trade_pr) OVER (PARTITION BY exchange,symbol ORDER BY event_tm RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as mov_avg_pr FROM tmp_curr_trade')
curr_ma_df.show(5,truncate=False)
# create current_ma temporary table
curr_ma_df.createOrReplaceTempView('curr_ma')

+------+--------+-----------------------+------------+--------+-----------------+
|symbol|exchange|event_tm               |event_seq_nb|trade_pr|mov_avg_pr       |
+------+--------+-----------------------+------------+--------+-----------------+
|SYMA  |NASDAQ  |2020-08-06 10:42:21.079|10          |78.93246|78.93245697021484|
|SYMA  |NASDAQ  |2020-08-06 12:00:29.595|20          |77.0967 |77.0967025756836 |
|SYMA  |NASDAQ  |2020-08-06 13:09:29.883|30          |78.31462|78.31462097167969|
|SYMA  |NASDAQ  |2020-08-06 14:27:08.62 |40          |75.84401|75.84400939941406|
|SYMA  |NASDAQ  |2020-08-06 15:39:00.929|50          |77.62613|77.62612915039062|
+------+--------+-----------------------+------------+--------+-----------------+
only showing top 5 rows



### Calculate Previous Day Trade Information
#### 30 minute moving average

In [24]:
# 30MA for previous days trade information
prev_ma=spark.sql('SELECT symbol,exchange,event_tm,event_seq_nb,last_mov_avg_pr FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY exchange,symbol ORDER BY event_tm DESC) as row FROM final_prev_ma) WHERE row=1')
prev_ma.show(truncate=False)
prev_ma.count()
prev_ma.createOrReplaceTempView('tmp_prev_ma')

+------+--------+-----------------------+------------+------------------+
|symbol|exchange|event_tm               |event_seq_nb|last_mov_avg_pr   |
+------+--------+-----------------------+------------+------------------+
|SYMA  |NASDAQ  |2020-08-05 21:33:59.339|99          |76.15560913085938 |
|SYMB  |NASDAQ  |2020-08-05 20:56:44.605|99          |36.75167465209961 |
|SYMC  |NASDAQ  |2020-08-05 21:41:00.998|99          |156.85586547851562|
|SYMA  |NYSE    |2020-08-05 21:23:39.176|99          |74.60467529296875 |
|SYMB  |NYSE    |2020-08-05 21:22:14.285|99          |34.249298095703125|
|SYMC  |NYSE    |2020-08-05 21:46:53.294|99          |159.7174835205078 |
+------+--------+-----------------------+------------+------------------+



#### Previous day close price

In [25]:
prev_day_close_pr=spark.sql('SELECT symbol,exchange,event_tm,event_seq_nb,trade_pr as last_trade_pr FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY exchange, symbol ORDER BY event_tm DESC) as row FROM tmp_prev_trade) WHERE row=1')
prev_day_close_pr.show(truncate=False)
#create close price tempview
prev_day_close_pr.createOrReplaceTempView('tmp_prev_close_pr')

+------+--------+-----------------------+------------+-------------+
|symbol|exchange|event_tm               |event_seq_nb|last_trade_pr|
+------+--------+-----------------------+------------+-------------+
|SYMA  |NASDAQ  |2020-08-05 21:40:23.197|100         |77.24676     |
|SYMB  |NASDAQ  |2020-08-05 21:03:26.585|100         |35.537262    |
|SYMC  |NASDAQ  |2020-08-05 21:49:28.543|100         |158.02032    |
|SYMA  |NYSE    |2020-08-05 21:30:19.773|100         |77.78611     |
|SYMB  |NYSE    |2020-08-05 21:27:28.314|100         |33.956287    |
|SYMC  |NYSE    |2020-08-05 21:52:38.607|100         |160.61949    |
+------+--------+-----------------------+------------+-------------+



In [26]:
#viewing the current quote table to understand data structure, info, and count
curr_quote.show(5,truncate=False)
curr_quote.count()

+----------+------+--------+-----------------------+------------+-------------------+---------+--------+---------+--------+
|trade_dt  |symbol|exchange|event_tm               |event_seq_nb|arrival_tm         |bid_pr   |bid_size|ask_pr   |ask_size|
+----------+------+--------+-----------------------+------------+-------------------+---------+--------+---------+--------+
|2020-08-06|SYMA  |NASDAQ  |2020-08-06 09:38:08.093|1           |2020-08-06 09:30:00|78.133705|100     |79.825165|100     |
|2020-08-06|SYMA  |NYSE    |2020-08-06 09:39:01.293|1           |2020-08-06 09:30:00|77.67913 |100     |78.437355|100     |
|2020-08-06|SYMA  |NASDAQ  |2020-08-06 09:46:05.163|2           |2020-08-06 09:30:00|76.52305 |100     |76.57241 |100     |
|2020-08-06|SYMA  |NYSE    |2020-08-06 09:47:20.398|2           |2020-08-06 09:30:00|76.53373 |100     |76.94425 |100     |
|2020-08-06|SYMA  |NASDAQ  |2020-08-06 09:52:14.798|3           |2020-08-06 09:30:00|78.74535 |100     |79.0928  |100     |
+-------

540

### Join Quote with current MA and spread 

In [27]:
# joining quote and ma 
joined_ma = spark.sql('SELECT q.symbol, q.exchange, q.event_tm, q.event_seq_nb, q.bid_pr, q.bid_size, q.ask_pr, q.ask_size, ma.trade_pr,ma.mov_avg_pr, ROW_NUMBER() OVER (PARTITION BY q.symbol, q.exchange, q.event_tm ORDER BY ma.event_tm DESC) as row_num FROM tmp_curr_quote as q LEFT JOIN curr_ma as ma ON q.symbol = ma.symbol AND q.exchange = ma.exchange AND ma.event_tm < q.event_tm')
joined_ma.createOrReplaceTempView('tmp_quote_ma')


In [28]:
# calculating and adding column for spead
joined_spread=spark.sql('SELECT q.symbol, q.exchange, q.event_tm, q.event_seq_nb, q.bid_pr, q.bid_size, q.ask_pr, q.ask_size, q.bid_pr-c.last_trade_pr as bid_spread, q.ask_pr-c.last_trade_pr as ask_spread, q.trade_pr as last_trade_pr, q.mov_avg_pr as last_mov_avg_pr FROM tmp_quote_ma as q LEFT JOIN tmp_prev_close_pr as c ON q.exchange=c.exchange AND q.symbol=c.symbol where row_num=1')
joined_spread.createOrReplaceTempView('tmp_quote_ma_spread')
joined_spread.count()

540

### Join Quote with previous close price and ma

In [29]:
joined_close=spark.sql('SELECT q.symbol, q.exchange, q.event_tm, q.event_seq_nb, q.bid_pr, q.bid_size, q.ask_pr, q.ask_size, q.bid_spread,q.ask_spread, t.last_trade_pr, m.last_mov_avg_pr FROM tmp_quote_ma_spread as q JOIN tmp_prev_close_pr as t ON q.exchange=t.exchange AND q.symbol=t.symbol JOIN tmp_prev_ma as m ON q.exchange=m.exchange AND q.symbol=m.symbol WHERE q.last_trade_pr IS NULL AND q.last_mov_avg_pr IS NULL')
joined_close.count()

54

### Union two Dataframes

In [35]:
#union current MA & spread dataframe with previous MA & price dataframe
final_df=joined_spread.union(joined_close)
#drop columns with null values
final_df=final_df.na.drop('any').orderBy('exchange','symbol','event_tm')
final_df.show(20,truncate=False)

+------+--------+-----------------------+------------+---------+--------+---------+--------+-----------+-----------+-------------+-----------------+
|symbol|exchange|event_tm               |event_seq_nb|bid_pr   |bid_size|ask_pr   |ask_size|bid_spread |ask_spread |last_trade_pr|last_mov_avg_pr  |
+------+--------+-----------------------+------------+---------+--------+---------+--------+-----------+-----------+-------------+-----------------+
|SYMA  |NASDAQ  |2020-08-06 09:38:08.093|1           |78.133705|100     |79.825165|100     |0.88694763 |2.5784073  |77.24676     |76.15560913085938|
|SYMA  |NASDAQ  |2020-08-06 09:46:05.163|2           |76.52305 |100     |76.57241 |100     |-0.7237091 |-0.6743469 |77.24676     |76.15560913085938|
|SYMA  |NASDAQ  |2020-08-06 09:52:14.798|3           |78.74535 |100     |79.0928  |100     |1.4985962  |1.8460388  |77.24676     |76.15560913085938|
|SYMA  |NASDAQ  |2020-08-06 09:58:51.806|4           |75.613625|100     |76.949776|100     |-1.6331329 |-0

### Load analytical data to storage

In [36]:
def load_analytical_data(df,date):
    base_path='/home/roger/SB/Guided_Capstone/output/analytical_data/'
    df.write.parquet(base_path+'{}/'.format(date))
    return 



In [38]:
load_analytical_data(final_df,'2020-08-06')